# 附录 10.2：工具使用

- [课程](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

# Override the MODEL_NAME variable in the IPython store to use Sonnet instead of the Haiku model
MODEL_NAME='anthropic.claude-3-sonnet-20240229-v1:0'
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

# Rewrittten to call Claude 3 Sonnet, which is generally better at tool use, and include stop_sequences
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages,
        system=system_prompt,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

---

## 课程

虽然一开始在概念上看起来有些复杂，但“工具使用”（即函数调用）其实很简单！你已经具备实现工具使用所需的全部技能，本质上就是“替换 + 链式提示”的组合。

在之前的“替换”练习中，我们把文本替换到提示中。到了“工具使用”，我们把“工具/函数的运行结果”替换进提示。Claude 并不能真正去调用或访问外部工具与函数。因此我们的做法是让 Claude：
1. 输出它想调用的工具名称及参数
2. 在工具执行期间暂停进一步的答案生成
3. 将工具结果附加回会话后，再次向 Claude 发起提示

函数调用之所以有用，是因为它扩展了 Claude 的能力，使其可以处理更复杂的多步骤任务。
你可以提供给 Claude 的函数示例包括：
- 计算器
- 字数统计
- SQL 数据库查询与数据检索
- 天气 API

要让 Claude 使用工具，你需要组合以下两部分：

1. 一个系统提示（system prompt），用于解释“工具使用”的概念，并提供它可用工具的详细清单
2. 一段控制逻辑，用来编排与执行 Claude 的工具调用请求

### 工具使用路线图

*本课讲授的是当前的工具使用格式；后续我们会更新与改进工具使用功能，包括：*
* *更精简的函数定义与调用格式*
* *更健壮的错误处理与边界情况覆盖*
* *与其他 API 的更紧密集成*
* *更可靠的执行与性能，尤其适用于复杂的工具使用任务*

### 示例

要在 Claude 中启用“工具使用”，我们需要先写一个系统提示。在这个专门用于工具使用的系统提示中，我们会告诉 Claude：
* 工具使用的基本前提与含义
* 如何调用并使用提供给它的工具
* 在当前场景中它可用工具的详细列表

以下是系统提示的第一部分，用来向 Claude 解释工具使用。这一部分具有通用性，适用于所有关于工具使用的提示场景。我们提供给 Claude 的工具调用结构（`<function_calls> [...] </function_calls>`）是 Claude 专门接受过训练的格式，建议你沿用该结构。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

以下是系统提示的第二部分：定义在本场景中 Claude 可以使用的具体工具。在这个示例中，我们将给 Claude 一个“计算器”工具，它接受三个参数：两个操作数和一个运算符。

随后我们把系统提示的两部分拼接起来。

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

现在我们可以给 Claude 一个需要使用 `calculator` 工具的问题。我们会在 `stop_sequences` 中使用 `</function_calls>`，以检测 Claude 何时调用函数并在该处停止生成。

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

接下来我们从 Claude 的函数调用输出里提取参数，并代表 Claude 实际执行该函数。

先定义函数的具体代码。

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

然后从 Claude 的函数调用响应中提取参数。若三个参数齐全，则运行计算器工具。

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

拿到结果后，我们需要把它格式化成 Claude 能够识别的“工具结果”结构，以便 Claude 理解该结果对应哪个工具。Claude 已经针对以下固定格式做过训练：
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

运行下面的单元，将上面的工具结果格式化为该结构。

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

最后一步，只需把该结果附加回同一会话消息链中，再发给 Claude 即可！

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

恭喜你完成了一条端到端的工具使用链！

那么，如果我们给 Claude 一个根本不需要使用该工具的问题，会发生什么呢？

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

成功！如你所见，在不需要时 Claude 会避免调用函数。

如果你想在不更改上述任何内容的情况下试验课程提示，请滚动到本课笔记本底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 10.2.1 - SQL](#exercise-1021---SQL)

### 练习 10.2.1 - SQL
在本练习中，你将为“查询与写入世界上最小的‘数据库’”编写一个工具使用提示。下面是已经初始化的“数据库”（其实就是一个字典）。

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

下面是用于读写该“数据库”的函数代码。

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

要完成本练习，请先像上文的 `system_prompt_tools_specific_tools` 那样定义一个系统提示。务必包含每个工具的名称与描述，以及每个函数的各个参数的名称、类型与说明。我们在下面提供了一个起始脚手架。

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

准备好后，可以在下方示例上试运行你的工具定义系统提示。直接运行下面这个单元即可！

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print Claude's response
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

如果你完成正确，函数调用消息应能正确调用 `add_user`、`add_product`、`get_user` 和 `get_product`。

加分项：再新建一些代码单元，编写参数解析代码。随后用 Claude 给出的参数去调用函数，观察调用后“数据库”的状态变化。

❓ 如果你想看到可能的解决方案，请运行下面的单元格！

In [ ]:
print(hints.exercise_10_2_1_solution)

### 恭喜！

恭喜你学会了工具使用与函数调用！如果想进一步了解搜索与 RAG，请前往最后一节附录。

---

## 示例练习场

这是一个区域，供你自由尝试本课展示的提示示例，并微调提示以观察它可能如何影响 Claude 的响应。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)